![](measure_CF_calls_logo.png)

# measure horseshoe bat calls
A package to automate measurements of Horseshoe bat calls - (or any similar sound!). The package is still under development - I would greatly appreciate any kind of feedback on how/whether it works on your data!

### What is a horseshoe bat call?
Horseshoe bats are a group of echolocating bat that emit calls that look like full or half staple pins. The logo above is a schematic spectrogram of a staplepin type call with the associated measures this package provides. 

This 'staplepin' can be divided into two parts: the constant frequency (CF) and frequency modulated (FM). The CF part is the 'flat' part of the staplepin, where the frequency emitted is constant. The FM parts of the call are where the frequency is either rising and/or descending. Not all calls have both rising and descending, and some calls may have only the rising FM or the descending FM. The relative durations of the Cf and FM are also known to change depending on the behavioural context at hand. 

### What does this package do?
It's all in the name. It measures various properties of a horseshoe bat call. 

Given a CF bat call, it segments out CF and FM components and calculates various measures. The FM and CF parts are separated in the waveform and various measurements are made on them including:

* CF peak frequency
* FM end frequency/ies
* CF and FM durations
* CF and FM rms and energy

### Who is this package useful for?
Mainly those studying bat calls, and in specific those analysing horseshoe bat calls. However, there may perhaps be a whole host of sounds which looks similar -- and so it might also be useful for the analysis of any sound that looks like a bat CF call!

### Why did you develop this package?
Measuring the sub-structure of horseshoe bat calls can be tiresome and error-prone when done manually. While the human eye can recognise the CF and FM parts of the call very well, segmenting it consistently is hard to do. This is because we rely on a spectrographic representation - the settings of which again influences how a call is segmented into FM and CF. Even existing automated methods in the field rely on spectrographic representations.  Moreover, to my knowledge, there aren't any  open-source tools developed specifically to measre call parameters for horseshoe bat calls. 


### Acknowledgements
The example data is a subset of audio recorded of *Rhinolophus mehelyi* and *Rhinolophus euryale* bats as they flew in and out of an evening roosting spot. Data collection was done by Neetash MR and Thejasvi Beleyur. Audio data annotation and selection done by Aditya Krishna. 

I thank Neetash MR, Diana Schoeppler & Holger R Goerlitz for their inputs in the development of this package.

### How do I use this package?

### Installation: 
This package works with both Python 2 and 3. 
Download the package using pip with:

```pip install measure_horseshoebat_calls ```

or

download this repo from Github,open up a terminal window, navigate to the folder,  and type:

```python setup.py install ```

### Usage:

The measure_horseshoe_bat_calls package takes in an audio file snippet with a *single* horseshoe bat call in it and outputs various measures of this call. Let us first load a manually selected part of a recording with a single horseshoe bat call in it. 

In [1]:
%matplotlib notebook

In [2]:
import measure_horseshoe_bat_calls

In [3]:
from  measure_horseshoe_bat_calls import measure_a_horseshoe_bat_call as measure
from measure_horseshoe_bat_calls import segment_horseshoebat_call as segment
from measure_horseshoe_bat_calls.view_horseshoebat_call import *




import soundfile as sf 
import scipy.signal as signal

In [4]:
# load a part of an audio file known to have a horseshoe bat call 
chosen_audio, fs = sf.read('figuring_out/real_data/matching_annotaudio_Aditya_2018-08-17_56_59.WAV')
chosen_audio_ups = signal.resample(chosen_audio, chosen_audio.size*2)
fs_up = fs*2

In [5]:
# view the waveform + spectrogram
visualise_call(chosen_audio_ups, fs_up)

<IPython.core.display.Javascript object>

(<matplotlib.axes._subplots.AxesSubplot at 0x10906c48>,
 <matplotlib.axes._subplots.AxesSubplot at 0x1397b908>)

### Segmenting the call from the silent background. 
Measuring temporal features such as call, CF and FM durations relies on an accurate and tight window selection that includes the call without too much of the background. Having more 'background' in the analyses will lead to inaccurate measurements of all parameters eg. duration and rms. Selecting a tight window can be simply done by hand, but also yields inconsistent results each time. Automating this process allows reproducibility across time. 

In [6]:
# get a tight window which only has the call. 
main_call_window = segment.segment_call_from_background(chosen_audio_ups, fs_up, 
                                                                          background_threshold=-20)

In [7]:
waveform, spec = visualise_call(chosen_audio_ups, fs_up)
waveform.plot(make_x_time(main_call_window, fs_up),main_call_window*np.max(chosen_audio_ups),'k')
waveform.plot(make_x_time(main_call_window, fs_up),main_call_window*np.min(chosen_audio_ups),'k')
spec.set_ylim(0,125000)
spec.plot(make_x_time(main_call_window, fs_up),main_call_window*120000,'w')

<IPython.core.display.Javascript object>

### Segmenting a horseshoe bat call into the FM and CF components:
To actually quantitatively characterise a horseshoe bat call we first need to define the constant frequency (CF) and frequency modulated (FM) parts of it. 

In [8]:
# choose only the call without any background
only_call = chosen_audio_ups[main_call_window]

In [9]:
cf, fm, info = segment.segment_call_into_cf_fm(only_call, fs_up, percentage_peak=0.99)

In [10]:
help(visualise_call)

Help on function visualise_call in module measure_horseshoe_bat_calls.view_horseshoebat_call:

visualise_call(audio, fs, **kwargs)
    Parameters
    ----------
    audio 
    fs 
    fft_size : integer>0, optional
    
    Returns
    -------
    a0, a1 : subplots



In [11]:
wavef, specg = visualise_call(only_call, fs_up,fft_size=512)
specg.set_ylim(0,125000)
wavef.plot(make_x_time(only_call, fs_up),cf*np.max(only_call),'k')
wavef.plot(make_x_time(only_call, fs_up),cf*np.min(only_call),'k')
wavef.plot(make_x_time(only_call, fs_up),fm*np.max(only_call),'r')
wavef.plot(make_x_time(only_call, fs_up),fm*np.min(only_call),'r')

specg.plot(make_x_time(cf, fs_up),cf*120000,'k',label='CF')
specg.plot(make_x_time(fm, fs_up),fm*80000,'r',label='FM')
plt.legend()

<IPython.core.display.Javascript object>

### Segmenting the call into CF and FM parts


In [12]:
sounds, measurements = measure.measure_hbc_call(only_call, fs_up, cf, fm)

In [13]:
measurements

{'call_duration': 0.021888,
 'call_energy': 7.255535997565814,
 'call_rms': 0.025748190189277226,
 'peak_frequency': 103938.23099415207,
 'cf_duration': 0.0186,
 'cf_energy': 7.072964600232841,
 'cf_rms': 0.0275777777926726,
 'cf_peak_frequency': 103978.49462365592,
 'downfm_duration': 0.001662,
 'downfm_energy': 0.11728999684960187,
 'downfm_rms': 0.011880370245560646,
 'downfm_terminal_frequency': 86040.91456077016,
 'upfm_duration': 0.001626,
 'upfm_energy': 0.06528140048337029,
 'upfm_rms': 0.008960855180383238,
 'upfm_terminal_frequency': 89790.8979089791,
 'terminal_frequency_threshold': -10}

### The overview figure and its subplots:

This is a viewing tool built to help the user decide whether the call has been segmented clearly. 

* Oscillogram: The waveform of the raw (blue) audio and the audio with CF portion removed (orange). 

* Manual selection : The exact waveform chosen by the user

* Fine selection of call : The 95%ile energy window of the manually selected window. This step narrows the time window the manual selection a little bit more. 

* Non-CF call : A spectrogram of the audio with CF portion filter out. 

* First FM : For horseshoe bat calls, this si typically an upward modulated FM segment. 

* Second FM : This is typically a downward FM segment. 

In [14]:
len(sounds)

2

### The output sounds 
The output sounds are typically: 1) the fine selection audio 2) audio with CF part removed 3) rising FM 4) rising FM. Depending on the number of FM portions detected in a call, there may only be 2,3 or 4 output sounds. The first two sounds are however always the fine audio selection and CF-filtered call. 

In [15]:
fine_selection, cf_filtered, fm_rising, fm_descending = sounds

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
plt.figure(figsize=(6,8))
plt.subplot(411)
plt.plot(fine_selection)
plt.subplot(412)
plt.plot(cf_filtered)
plt.subplot(413)
plt.plot(fm_rising)
plt.subplot(414)
plt.plot(fm_descending)

In [ ]:
# print the measurements
measurements

### Choosing the right parameter settings for *your* bat calls!
The segmentation is of course affected by the parameter settings given to ```measure_hbc_call ```. The current default set may be good for many situations - but you may want better! 

### How does the call segmentation work?
The call segmentation works on the fact that horseshoe bat calls have most of their energy in one part of their call, the constant frequency (CF) part. The CF part of a call is typically flanked by two FMs. ```segment_call_into_cf_fm ``` goes through the following steps for every input audio. 

* 1. Separate call from background by running a continuous wavelet transform on the audio. The coefficients of all scales with centre frequencies in the signal range are summed up. This forms a kind of signal-band 'energy-profile' which tracks the onset and offset of the signal with high temporal resolution. 

* 2. Separate FM and CF by forming two versions of the call filtered at the same threshold frequency: i) low-pass filtered ii) high-pass filtered. The threshold frequency for low and high pass filtering is user-definable, and set by default to 0.99 of the detected peak frequency.

* 3. A dB rms profile of the low and high pass filtered audio is calculated. The main idea is that the low pass dBrms profile will have peaks towards the edges (where the FM sweeps are), while the highpass dBrms profile will have a broad bump in the centre. How to separate out the two reliably? The separation is done by taking a difference profile. eg. The portions of the $lowpass_{dB \:rms}-highpass_{dB \:rms}$ difference profile which are $\geq 1$ are the FM parts, while the portions of the $highpass_{dB \:rms}-lowpass_{dB \:rms} >0 $ are the CF parts.


### Feedback and Contributions Welcome!! 
### Thejasvi Beleyur,  Acoustic and Functional Ecology Group, Max Planck Institute for Ornithology, Seewiesen

-- Last updated 2020-02-14 15:30 CET